In [1]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import cv2

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [2]:
dosya=r'C:\Users\serde\OneDrive\Masaüstü\captioning'

In [3]:
catalog= pd.read_csv(r"C:\Users\serde\OneDrive\Masaüstü\captioning\catalog.txt", sep="|")
catalog

,Image,Caption
0,194508109.jpg,mock neck embroidery suede sweatshirt .
1,188778349.jpg,luxe double zip hooded jacket .
2,188977857.jpg,citizens humanity high rise rocket hem jean .
3,194942557.jpg,suede tie short boots .
4,194941874.jpg,cloth travel school backpack .
...,...,...
49677,190070434.jpg,metal rhinestones pu leather shoulder bag .
49678,177433619.jpg,burberry kensington leather trim trench coat .
49679,186517066.jpg,lace top chunky booties .
49680,186779723.jpg,christian louboutin kate patent-leather pumps .


In [4]:
catalog.drop_duplicates(inplace=True)

In [5]:
catalog

,Image,Caption
0,194508109.jpg,mock neck embroidery suede sweatshirt .
1,188778349.jpg,luxe double zip hooded jacket .
2,188977857.jpg,citizens humanity high rise rocket hem jean .
3,194942557.jpg,suede tie short boots .
4,194941874.jpg,cloth travel school backpack .
...,...,...
49675,190070229.jpg,belted high collar coat leather sleeve .
49676,189165560.jpg,gucci snake leather pumps .
49677,190070434.jpg,metal rhinestones pu leather shoulder bag .
49678,177433619.jpg,burberry kensington leather trim trench coat .


In [6]:
#pd.DataFrame(catalog).to_csv(r'C:\Users\serde\OneDrive\Masaüstü\catalog.txt',sep="|", index=None) 

In [7]:
train_df= pd.read_json(r"C:\Users\serde\OneDrive\Masaüstü\bitirme\polyvore\train_no_dup.json")
train_df

,name,views,items,image,likes,date,set_url,set_id,desc
0,Casual,8743,"[{'index': 1, 'name': 'mock neck embroidery su...",http://ak1.polyvoreimg.com/cgi/img-set/cid/214...,394,One month,http://www.polyvore.com/casual/set?id=214181831,214181831,A fashion look from January 2017 by beebeely-l...
1,Being a Vans shoe model with Luke. Idk about t...,188,"[{'index': 1, 'name': 'nirvana distressed t-sh...",http://ak1.polyvoreimg.com/cgi/img-set/cid/120...,9,Two years,http://www.polyvore.com/being_vans_shoe_model_...,120161271,A fashion look from April 2014 featuring destr...
2,These Chanel bags is a bad habit .x,562,"[{'index': 1, 'name': 'monki singlet', 'price'...",http://ak1.polyvoreimg.com/cgi/img-set/cid/143...,32,Two years,http://www.polyvore.com/these_chanel_bags_is_b...,143656996,12.19.14
3,Avenger/Supernatural prp,2613,"[{'index': 1, 'name': 'tops', 'price': 18.0, '...",http://ak2.polyvoreimg.com/cgi/img-set/cid/186...,88,One year,http://www.polyvore.com/avenger_supernatural_p...,186627934,A fashion look from January 2016 by alyssaclai...
4,Boho (39),62,"[{'index': 1, 'name': 'yoins leather sexy v-ne...",http://ak2.polyvoreimg.com/cgi/img-set/cid/206...,3,5 months,http://www.polyvore.com/boho_39/set?id=206969379,206969379,A fashion look from August 2016 by kate-goida ...
...,...,...,...,...,...,...,...,...,...
17311,2.15.17,183,"[{'index': 1, 'name': 'raey deep v-neck silk c...",http://ak1.polyvoreimg.com/cgi/img-set/cid/216...,117,4 days,http://www.polyvore.com/15.17/set?id=216801059,216801059,A fashion look from February 2017 by kianahall...
17312,tweed,2428,"[{'index': 1, 'name': 'michael kors printed bo...",http://ak1.polyvoreimg.com/cgi/img-set/cid/190...,382,One year,http://www.polyvore.com/tweed/set?id=190488700,190488700,A fashion look from February 2016 by drigomes ...
17313,TV Style: Pretty Little Liars,2184,"[{'index': 1, 'name': 'color block fit flare m...",http://ak2.polyvoreimg.com/cgi/img-set/cid/187...,398,One year,http://www.polyvore.com/tv_style_pretty_little...,187504514,A fashion look from January 2016 by vanjazivad...
17314,60-Second Style: Work Wear,3147,"[{'index': 1, 'name': 'patchwork fit flare dre...",http://ak2.polyvoreimg.com/cgi/img-set/cid/211...,440,Three months,http://www.polyvore.com/60-second_style_work_w...,211085207,A fashion look from November 2016 by beebeely-...


In [8]:
count=0
while count<len(train_df):
    x=0
    y=len(train_df['items'][count])
    while x<y:
        id=train_df['items'][count][x]['image'].split("id=",1)[1]
        train_df['items'][count][x]['image']=id+".jpg"
        x+=1
    count+=1  

In [9]:
IMAGE_PATH = r"C://Users//serde//images//"

In [10]:
model = VGG16()

model=Model(inputs=model.inputs, outputs=model.layers[-2].output)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
train_df['items'][0][0]["image"].split('.')[0]

'194508109'

In [12]:
features = {}

m=0
while m<len(train_df):
    n=0
    l=len(train_df['items'][m])
    while n<l:
        filename = IMAGE_PATH + train_df['items'][m][n]["image"]
        image_id= train_df['items'][m][n]["image"].split('.')[0]
        isFile = os.path.isfile(filename)
        name=train_df['items'][m][n]["name"]
        if isFile==True and name!="":
            image = load_img(filename, target_size=(224, 224))
            image = img_to_array(image)
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            features[image_id] = feature

        n+=1
    m+=1 


KeyboardInterrupt: 

In [ ]:
#pickle.dump(features, open(os.path.join(dosya, 'features.pkl'), 'wb'))

In [13]:
with open(os.path.join(dosya, 'features.pkl'), 'rb') as f:
    features = pickle.load(f)

In [14]:
with open(os.path.join(dosya, 'caption_catalog.txt'), 'r') as f:
    next(f)
    captions_doc = f.read()

In [15]:
captions_doc

"194508109.jpg|mock neck embroidery suede sweatshirt .\n188778349.jpg|luxe double zip hooded jacket .\n188977857.jpg|citizens humanity high rise rocket hem jean .\n194942557.jpg|suede tie short boots .\n194941874.jpg|cloth travel school backpack .\n103135174.jpg|rag bone rock w/ black skinny jeans .\n107307153.jpg|vans authentic black mono trainers .\n124815566.jpg|joy denim jacket .\n125624471.jpg|topshop moto joni high rise skinny jeans .\n125451606.jpg|rag bone floppy brim fedora .\n178357420.jpg|yoins leather sexy v-neck sleeveless crop top .\n135914227.jpg|solid color long sleeve irregular blazer .\n180944840.jpg|alice+olivia floral pattern a-line skirt .\n170644849.jpg|gold boho turquoise leaf tassel earrings .\n191683321.jpg|etro heart locket necklace .\n165020476.jpg|sophia webster leather butterfly flats .\n141342171.jpg|michael michael kors mini selma crossbody bag .\n180707667.jpg|isabel marant alpaca blend jumper .\n177509842.jpg|yoins light blue gradient color hole denim s

In [17]:
mapping = {}

for line in tqdm(captions_doc.split('\n')):
    tokens = line.split('|')
    image_id, caption = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    caption = " ".join(caption)
    if image_id not in mapping:
        mapping[image_id] = []
    mapping[image_id].append(caption)

In [18]:
mapping

{'194508109': ['mock neck embroidery suede sweatshirt .'],
 '188778349': ['luxe double zip hooded jacket .'],
 '188977857': ['citizens humanity high rise rocket hem jean .'],
 '194942557': ['suede tie short boots .'],
 '194941874': ['cloth travel school backpack .'],
 '103135174': ['rag bone rock w/ black skinny jeans .'],
 '107307153': ['vans authentic black mono trainers .'],
 '124815566': ['joy denim jacket .'],
 '125624471': ['topshop moto joni high rise skinny jeans .'],
 '125451606': ['rag bone floppy brim fedora .'],
 '178357420': ['yoins leather sexy v-neck sleeveless crop top .'],
 '135914227': ['solid color long sleeve irregular blazer .'],
 '180944840': ['alice+olivia floral pattern a-line skirt .'],
 '170644849': ['gold boho turquoise leaf tassel earrings .'],
 '191683321': ['etro heart locket necklace .'],
 '165020476': ['sophia webster leather butterfly flats .'],
 '141342171': ['michael michael kors mini selma crossbody bag .'],
 '180707667': ['isabel marant alpaca blend

In [19]:
def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            caption = captions[i]
            caption = caption.lower()
            caption = caption.replace('[^A-Za-z]', '')
            caption = caption.replace('\s+', ' ')
            #tek harf olan kelimeleri siliyoruz çünkü bir şey ifade etmiyor.
            caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
            #start ve end ekliyoruz ki sonraki kelimeyi tahmin etmeye çalışırken cümlenin bittiğini anlasın
            captions[i] = caption

In [20]:
mapping["174958652"]

['chloÃ© printed silk blouse .']

In [21]:
clean(mapping)

In [22]:
mapping["174958652"]

['startseq chloã© printed silk blouse endseq']

In [23]:
all_captions = []
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [24]:
len(all_captions)

31224

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
#sözlük boyutunu bulmak için unique kelimeleri sayıyoruz. word_index'teki her eleman bir unique word
vocab_size = len(tokenizer.word_index) + 1

In [26]:
vocab_size

2503

In [27]:
#en uzun caption
max_length = max(len(caption.split()) for caption in all_captions)
max_length

27

In [28]:
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [29]:
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0 #gerekli batch sayısına eriştik mi?
    while 1:
        for key in data_keys:
            
            n += 1 
            captions = mapping[key]
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]
                
                #sonraki kelimeyi tahmin etmesi için
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

In [32]:
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)


decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [33]:
batch_size = 32
steps = len(train) // batch_size

generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
fitted =model.fit(generator, epochs=20, steps_per_epoch=steps, verbose=1)

878/878 [==============================] - 294s 334ms/step - loss: 2.0122


In [ ]:
#model.save(dosya+'/model.h5')